In [7]:
import random, copy
import cv2
import numpy as np
import random
from skimage.metrics import structural_similarity as ssim

In [8]:
# def objective_function(position):
#     # Create binary mask based on threshold value
#     threshold_value = position
#     _, mask = cv2.threshold(img, threshold_value, 255, cv2.THRESH_BINARY)
    
#     # Compute percentage of foreground pixels in mask
#     n_pixels = mask.flatten().shape[0]
#     n_foreground_pixels = np.count_nonzero(mask == 255)
#     if n_pixels > 0:
#         percent_foreground_pixels = n_foreground_pixels / n_pixels
#     else:
#         percent_foreground_pixels = 0

#     # Return negative of percentage of foreground pixels as fitness value
#     return percent_foreground_pixels

In [67]:
def objective_function(indivudual):
    # Create binary mask based on threshold value
    ss=[]
    for i in range(0,POPULATION_SIZE,8):
        res = int("".join(str(x) for x in indivudual[i:i+8]), 2)
#         print(res)
        threshold_value = res
        _, mask = cv2.threshold(img, threshold_value, 255, cv2.THRESH_BINARY)
        ss.append(ssim(img,mask))
    return ss

In [10]:
### POPULATION LEVEL ###
def gpso_search():
    population = create_pop()
    personal_best_population = copy.copy(population)
    global_best, global_best_fitness = get_best_pop(personal_best_population)
    #print(global_best_fitness)
    for _ in range(NUM_GENERATIONS):
       # print(population,'\n')
        population = convex_combination_pop(population, personal_best_population, [global_best]*POPULATION_SIZE)
        population = mutation_pop(population)
        personal_best_population = selection_pop(personal_best_population, population)
        global_best, global_best_fitness = get_best_pop(personal_best_population)
        #print(global_best_fitness)
    return (global_best, global_best_fitness) 

In [82]:
### INDIVIDUAL LEVEL ###

def create_pop():
    return [ create_ind() for _ in range(POPULATION_SIZE) ]
def get_best_pop(population):
    a=[]
    for ind in population:
        a.append(objective_function(ind))
    best_ind = max(a, key=evaluate_ind)
    for i in range(POPULATION_SIZE):
        if(best_ind==a[i]):
            return population[i],evaluate_ind(best_ind)

# def get_best_pop(population):
#     best_ind = max(population, key=evaluate_ind)
#    # print(best_ind)
# #     a=[objective_function(best_inds) for best_inds in best_ind]
#     a=[image_entropy(best_inds) for best_inds in best_ind]
#     return best_ind,evaluate_ind(a)

def convex_combination_pop(pop1, pop2, pop3):
    return [ convex_combination_ind([ind1, ind2, ind3]) for (ind1, ind2, ind3) in zip(pop1, pop2, pop3) ]

def mutation_pop(population):
    return [ mutation_ind(individual) for individual in population ]

def selection_pop(pop1, pop2):
    return [ max(ind1, ind2, key=evaluate_ind) for (ind1, ind2) in zip(pop1, pop2) ]



In [32]:
### REPRESENTATION LEVEL ###

def create_ind():
    return [ random.randint(0,1) for _ in range(INDIVIDUAL_SIZE) ]

def evaluate_ind(individual): #one_max
    return sum(individual)

def convex_combination_ind(mating_pool):
    transposed_mating_pool=zip(*mating_pool)
    return map(random.choice, transposed_mating_pool)

def mutation_ind(individual):
    return [ 1-bit if random.random()<1.0/INDIVIDUAL_SIZE else bit for bit in individual ]


In [83]:
a=[]
population=create_pop()
for ind in population:
    a.append(objective_function(ind))
best_ind = max(a, key=evaluate_ind)
for i in range(POPULATION_SIZE):
    if(best_ind==a[i]):
        print(population[i])    
print(best_ind,evaluate_ind(best_ind))

[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1]
[0.6396921241418356, 0.6264898533777956, 0.6329683448833509, 0.6526280638404884] 2.5517783862434706


In [72]:
for i in range(POPULATION_SIZE):
    if(best_ind==a[i]):
        print(population[i])

[0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1]


In [102]:
NUM_GENERATIONS = 50   
POPULATION_SIZE = 25
INDIVIDUAL_SIZE = 32
img = cv2.imread('1.jpg',0)
imgC=cv2.imread('1.jpg')

In [84]:
# img_gbr = cv2.imread('1.jpg')
# img_gbr = cv2.cvtColor(img_gbr, cv2.COLOR_BGR2HSV) 

# img = img_gbr[:,:,2]
### EXPERIMENTS ###


global_best2, global_best_fitness2=gpso_search()

In [85]:
print(global_best2)

[0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]


In [87]:
global_best_fitness2

2.4904105439580313

In [167]:
for i in range(0,POPULATION_SIZE,8):
    res = int("".join(str(x) for x in global_best2[i:i+8]), 2)
    threshold_value = res
    _, mask = cv2.threshold(img, threshold_value, 255, cv2.THRESH_BINARY)
    segmented_image = cv2.bitwise_and(img, img, mask=mask)
    segmented = np.copy(imgC)
    for i in range(imgC.shape[0]):
        for j in range(imgC.shape[1]):
            if mask[i][j]==0:
                segmented[i,j,:]=[0.0,255.0,0.0]
    cv2.imshow("Segmented Image_"+str(res), segmented_image)
    cv2.imshow("Segmented Image"+str(res), segmented)
    cv2.imshow("mask"+str(res),mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [164]:
segmented = np.copy(imgC)
cv2.minMaxLoc(mask)

(0.0, 255.0, (610, 136), (0, 0))

In [165]:
for i in range(imgC.shape[0]):
    for j in range(imgC.shape[1]):
        if mask[i][j]==0:
            segmented[i,j,:]=[0.0,255.0,0.0]


In [166]:
cv2.imshow("Segmented Image"+str(res), segmented)
cv2.imshow("mask"+str(res),mask)
cv2.waitKey(0)
cv2.destroyAllWindows()